In [ ]:
def get_fidelity(circuit, param_length):
    nshot=10000
    nparam=4000
    fidelity=[]    
    for x in range(nparam):
        params = 2*pi*random(param_length)
        res = circuit(params)

        if '0' in count:
            ratio=count['0']/nshot
        else:
            ratio=0
        fidelity.append(ratio)
    return fidelity